Source: https://www.nasdaq.com/symbol/fb/option-chain?expir=stan&dateindex=-1
Starter code from https://realpython.com/python-web-scraping-practical-introduction/

In [5]:
from bs4 import BeautifulSoup
from web_util import *

In [6]:
## get stock price
def getStockLatestPrice(html):    
    return float(html.find(id="qwidget_lastsale").string.replace("$", ""))

In [16]:
import re
from datetime import datetime
import pandas as pd
import numpy as np

def getOptionData(html):
    chart_div = html.find("div", class_="OptionsChain-chart")
    chart_headers = []
    for th in chart_div.find_all("th"):
        if th.find("a"):
            chart_headers.append(re.search(r'\".+\"', th.find("a").text)[0].replace('\"', ''))
        else:
            chart_headers.append(th.text)
    ## update for duplicate column names
    seen = []
    for c, col in enumerate(chart_headers):
        if col == 'Root':
            chart_headers[c] = 'Ticker'
        if col in seen:
            chart_headers[c] = col + "_2"
        else:
            seen.append(col)
    
    rows = chart_div.findAll("tr")
    rowlist = []
    for i, row in enumerate(rows):
        ## Find rows that have option price data
        ## These rows would have 2 <td>s with links to the option price details for the corresponding call/put, expiry and strike
        results = row.find_all(href=re.compile("option-chain"))
        
        if len(results) > 0:
            #expiry_date = datetime.strptime(results[0].text, '%b %d, %Y')            
            rowlist.append([td.text for td in row.find_all("td")])
    df = pd.DataFrame(rowlist)
    df.columns = chart_headers
    return df

In [18]:
## We're interested in doing covered calls, so just get bid prices for calls (Bid column), Strike, and Calls (expiry date) columns
def getDataForCoveredCalls(html):
    return getOptionData(html)[["Ticker", "Calls", "Bid", "Strike"]]


In [29]:
def processTicker(ticker):
    ## given a ticker symbol, get its stock and option pricing and return a dataframe 
    ## with all the relevant fields and calculations
    base_url = "https://www.nasdaq.com/symbol/{}/option-chain".format(ticker)
    url = base_url + "?expir=stan&dateindex=-1"
    page_content = simple_get(url)
    html = BeautifulSoup(page_content, 'html.parser')
    
    """ calculate yield and annualized yield
    based on the number of weeks to option expiry
    round up the number of weeks to expiry to make the annualized yield more conservative 
    Calculate what the yield would be if the option ends up being exercised or out of money:
    Annual yield if exercised: 
        annualized yield = (negative_premium + yield_to_expiry) * 52 / weeks_to_expiry 
    Out of money annualized yield
    Assume annualized yield to be the minimum of in-the-money and out-of-money annualized yields
    """
    latestprice = getStockLatestPrice(html)
    min_yield = 0.2
    df = getDataForCoveredCalls(html)
    df.loc[:, "Calls"] = pd.to_datetime(df["Calls"])
    df.loc[:, "WeeksToExpiry"] = np.ceil((df["Calls"] - datetime.today())/np.timedelta64(1, 'W')).astype(int)
    df.loc[:, "Bid"] = df["Bid"].replace("", "0").astype(float)
    df.loc[:, "Strike"] = df["Strike"].astype(float)
    df.loc[:, "StrikeToCurPricePremium%"] = np.floor(df["Strike"]*100/latestprice - 100)
    df.loc[:, "YieldToExpiry%"] = (df["Bid"] * 100/latestprice).astype(int)
    df.loc[:, "OOMAnnualizedYield%"] = (df.loc[:, "YieldToExpiry%"] * 52 / df["WeeksToExpiry"]).astype(int)
    df.loc[:, "ITMAnnualizedYield%"] = ((df["StrikeToCurPricePremium%"] + df["YieldToExpiry%"]) * 52 / df["WeeksToExpiry"]).astype(int)
    df.loc[:, "AnnualizedYield%"] = df[["OOMAnnualizedYield%", "ITMAnnualizedYield%"]].min(axis=1)
    df.loc[:, "Latest Price"] = latestprice
    return df

In [30]:
ticker_list = ['dis', 'de', 'cat', 'dal', 'jpm', 'v', 'clx', 'fb', 'aapl', 'luv', 'srg', 'syf', 'ge']
dfs = []
for ticker in ticker_list:
    dfs.append(processTicker(ticker))

In [36]:
min_yield = 0.25
combined_df = pd.concat(dfs, axis=0)
combined_df[combined_df["AnnualizedYield%"] >= min_yield*100].sort_values(by="AnnualizedYield%", ascending=False)

,Ticker,Calls,Bid,Strike,WeeksToExpiry,StrikeToCurPricePremium%,YieldToExpiry%,OOMAnnualizedYield%,ITMAnnualizedYield%,AnnualizedYield%,Latest Price
12,DAL,2019-04-18,1.20,58.0,2,0.0,2,52,52,52,57.71
2,GE,2019-04-18,0.27,9.5,2,0.0,2,52,52,52,9.49
1,GE1,2019-04-18,0.83,9.0,2,-6.0,8,208,52,52,9.49
17,FB,2019-05-17,7.60,175.0,6,0.0,4,34,34,34,174.93
8,GE1,2019-06-21,1.20,9.0,11,-6.0,12,56,28,28,9.49
10,GE1,2019-06-21,0.63,10.0,11,5.0,6,28,52,28,9.49
14,CAT,2019-04-18,1.89,140.0,2,0.0,1,26,26,26,139.82
15,CAT,2019-04-18,1.45,141.0,2,0.0,1,26,26,26,139.82
13,DIS,2019-04-18,1.58,116.0,2,0.0,1,26,26,26,114.96
6,GE,2019-05-17,0.37,10.0,6,5.0,3,26,69,26,9.49
